# Azure AI-ügynökök Model Context Protocol (MCP) támogatással - Python

Ez a jegyzetfüzet bemutatja, hogyan használhatók az Azure AI-ügynökök Model Context Protocol (MCP) eszközökkel Pythonban. Megmutatja, hogyan hozhatunk létre egy intelligens ügynököt, amely külső MCP szervereket (például Microsoft Learn) használhat a képességek bővítésére kulcs nélküli hitelesítéssel.


## Szükséges Python csomagok telepítése

Először telepítenünk kell a szükséges Python csomagokat:
- **azure-ai-projects**: Az Azure AI Projects SDK alapcsomagja
- **azure-ai-agents**: Azure AI Agents SDK az ügynökök létrehozásához és kezeléséhez
- **azure-identity**: Kulcs nélküli hitelesítést biztosít DefaultAzureCredential használatával
- **mcp**: Model Context Protocol implementáció Pythonhoz


## Kulcs nélküli hitelesítés előnyei

Ez a jegyzetfüzet bemutatja a **kulcs nélküli hitelesítést**, amely számos előnnyel jár:
- ✅ **Nincs kezelendő API-kulcs** - Azure-alapú identitáshitelesítést használ
- ✅ **Fokozott biztonság** - Nincsenek titkok a kódban vagy konfigurációs fájlokban tárolva
- ✅ **Automatikus hitelesítő adatok forgatása** - Az Azure kezeli a hitelesítő adatok életciklusát
- ✅ **Szerepkör-alapú hozzáférés-vezérlés** - Azure RBAC-t használ a finomhangolt jogosultságokhoz
- ✅ **Többkörnyezetes támogatás** - Zökkenőmentesen működik fejlesztési és éles környezetben is

A `DefaultAzureCredential` automatikusan kiválasztja a legjobb elérhető hitelesítési forrást:
1. **Kezelt identitás** (ha az Azure-ban fut)
2. **Azure CLI** hitelesítő adatok (helyi fejlesztés során)
3. **Visual Studio** hitelesítő adatok
4. **Környezeti változók** (ha konfigurálva vannak)
5. **Interaktív böngésző** hitelesítés (végső megoldásként)


## Kulcs nélküli hitelesítés beállítása

**Előfeltételek a kulcs nélküli hitelesítéshez:**

### Helyi fejlesztéshez:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure környezetekhez:
- Engedélyezze a **Rendszer által hozzárendelt Kezelt Identitást** az Azure erőforrásán
- Adjon megfelelő **RBAC szerepköröket** a kezelt identitáshoz:
  - `Cognitive Services OpenAI User` az Azure OpenAI hozzáféréshez
  - `AI Developer` az Azure AI Projektek hozzáféréshez

### Környezeti változók (Opcionális):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Nincs szükség API kulcsokra vagy kapcsolati karakterláncokra!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Szükséges könyvtárak importálása

Importáld a szükséges Python modulokat:
- **os, time**: Alapvető Python könyvtárak környezeti változókhoz és késleltetésekhez
- **AIProjectClient**: Fő kliens az Azure AI Projektekhez
- **DefaultAzureCredential**: Kulcs nélküli hitelesítés az Azure szolgáltatásokhoz
- **MCP-hez kapcsolódó osztályok**: MCP eszközök létrehozásához és kezeléséhez, valamint jóváhagyások kezeléséhez


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## MCP szerver beállításainak konfigurálása

Állítsd be az MCP szerver konfigurációját környezeti változók használatával, amelyek alapértelmezett értékekkel rendelkeznek:
- **MCP_SERVER_URL**: Az MCP szerver URL-je (alapértelmezés szerint a Microsoft Learn API)
- **MCP_SERVER_LABEL**: Egy címke az MCP szerver azonosításához (alapértelmezés szerint "mslearn")

Ez a megközelítés lehetővé teszi a rugalmas konfigurációt különböző környezetekben.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI Projekt kliens létrehozása (Kulcs nélküli hitelesítés)

Inicializálja az Azure AI Projekt klienst **kulcs nélküli hitelesítéssel**:
- **endpoint**: Az Azure AI Foundry projekt végpont URL-je
- **credential**: A `DefaultAzureCredential()` használata biztonságos, kulcs nélküli hitelesítéshez
- **Nincs szükség API kulcsokra**: Automatikusan felismeri és használja a legjobb elérhető hitelesítési módszert

**Hitelesítési folyamat:**
1. Ellenőrzi a Kezelt Identitást (Azure környezetekben)
2. Visszaáll az Azure CLI hitelesítő adatokra (helyi fejlesztéshez)
3. Szükség esetén más elérhető hitelesítési forrásokat használ

Ez a megközelítés kiküszöböli az API kulcsok vagy kapcsolati karakterláncok kezelésének szükségességét a kódban.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## MCP Eszközdefiníció Létrehozása

Hozzon létre egy MCP eszközt, amely csatlakozik a Microsoft Learn MCP szerverhez:
- **server_label**: Az MCP szerver azonosítója
- **server_url**: Az MCP szerver URL végpontja
- **allowed_tools**: Opcionális lista az engedélyezett eszközök korlátozására (üres lista esetén minden eszköz engedélyezett)

Ez az eszköz lehetővé teszi az ügynök számára, hogy hozzáférjen a Microsoft Learn dokumentációhoz és erőforrásokhoz.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Ügynök létrehozása és beszélgetés végrehajtása (Kulcs nélküli munkafolyamat)

Ez az átfogó szakasz bemutatja a teljes **kulcs nélküli ügynök munkafolyamatot**:

1. **AI ügynök létrehozása**: Állítsd be az ügynököt GPT-4.1 nano modellel és MCP eszközökkel
2. **Szál létrehozása**: Hozz létre egy beszélgetési szálat a kommunikációhoz
3. **Üzenet küldése**: Kérdezd meg az ügynököt az Azure OpenAI és az OpenAI közötti különbségekről
4. **Eszközjóváhagyások kezelése**: Automatikusan hagyd jóvá az MCP eszköz hívásokat, amikor szükséges
5. **Végrehajtás nyomon követése**: Kövesd nyomon az ügynök előrehaladását, és kezeld a szükséges lépéseket
6. **Eredmények megjelenítése**: Mutasd meg a beszélgetés és az eszközhasználat részleteit

**Kulcs nélküli funkciók:**
- ✅ **Nincsenek beágyazott titkok** - Minden hitelesítés az Azure identitás által történik
- ✅ **Alapértelmezett biztonság** - Szerepkör-alapú hozzáférés-vezérlést használ
- ✅ **Egyszerűsített telepítés** - Nincs szükség hitelesítő adatok kezelésére
- ✅ **Auditbarát** - Minden hozzáférés az Azure identitáson keresztül nyomon követhető

Az ügynök MCP eszközöket fog használni a Microsoft Learn források eléréséhez teljes biztonsággal, API kulcs kezelés nélkül.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
